In [1]:
import pdfplumber
import re
import pandas as pd
import os
import argparse


In [13]:
def extract_ticket_info(pdf_path):
    data = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                departure_location = re.search(r'Hinfahrt:\n(?:Gültig ab: .+)?\n([^,]+),', text)
                arrival_location = re.search(r'Freiburg\(Breisgau\) Hbf', text)
                departure_time = re.search(r'Halt Datum Zeit Gleis Produkte Reservierung\n([^ ]+ \d{2}:\d{2})', text)
                arrival_time = re.search(r'Freiburg\(Breisgau\) Hbf .+ \d{2}:\d{2}', text)
                cost = re.search(r'ICE Fahrkarte .+ (\d+,\d{2}€)', text)
                
                if departure_location and arrival_location and departure_time and arrival_time and cost:
                    data.append({
                        'Departure Location': departure_location.group(1),
                        'Arrival Location': arrival_location.group(0),
                        'Departure Time': departure_time.group(1),
                        'Arrival Time': arrival_time.group(0).split()[-1],
                        'Cost': cost.group(1)
                    })

    df = pd.DataFrame(data)
    return df

In [14]:
pdf_path = 'C:\\Users\\freed\\develop\\DB_OEBB_ticket\\new_ticket\\Ticket_298345646706_13.11.2024.pdf'
df = extract_ticket_info(pdf_path)

In [15]:
data = []
df = pd.DataFrame(data, columns=['Travel Date', 'Train', 'Cost'])
df

,Travel Date,Train,Cost


In [16]:
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if text:
            match = re.search(r'e\nGültigkeit: (\d{2}.\d{2}.\d{4}) 00:00 Uhr', text)
            if match:
                travel_date = match.group(1)
            
            match = re.search(r'\nEinfache Fahrt (.*?)\nVia:', text)
            if match:
                train = match.group(1)
            
            match = re.search(r'\nGesamtpreis (.*?)€', text, re.DOTALL)
            if match:
                cost = match.group(1)
                
df.loc[0] = [travel_date , train ,  cost]
df

,Travel Date,Train,Cost
0,13.11.2024,Köln Hbf Villach Hbf,"86,43"


In [17]:
text

'Online-Ticket\nCIV 1080\nICE Fahrkarte\nGültigkeit: 13.11.2024 00:00 Uhr bis 15.11.2024 03:00 Uhr\nSie können alle Züge nutzen, die auf Ihrer Fahrkarte angegeben sind. Für Züge des Nahverkehrs\n(z.B. RE, RB, S) besteht keine Zugbindung.\nSparpreis Europa (Einfache Fahrt)\nKlasse 2.Klasse\nReisende 1 Person (27-64 Jahre) mit 1 BC25 (32 Jahre)\n1 Fahrrad\nEinfache Fahrt Köln Hbf Villach Hbf\nVia: <1080>KO*(F*WUE*N/KA*S*UL)*M*SALZ\n<1181>Hallein*Spittal-Millstsee\nZugbindung ICE 1621, 07:53 Uhr am 13.11.2024\nEC 115, 14:17 Uhr am 13.11.2024\nEC 115, 16:12 Uhr am 13.11.2024\nSie können Ihre Fahrkarte bis einschließlich 12.11.2024 gegen 10,00 € Entgelt stornieren. Über den\nverbleibenden Betrag erhalten Sie einen Gutschein. Danach ist eine Stornierung ausgeschlossen.\nGesamtpreis 86,43 €. Gebucht am 26.10.2024 um 10:11 Uhr.\nDieses Dokument ist nicht vorsteuerabzugsfähig.\n!nekcink\nthcin\nettib\nedocraB\nZangenabdruck\nJohn Doe\nAuftragsnummer: 298345646706\nIhre Reiseverbindung und Reser

In [18]:
def extract_ticket_info_from_folder(folder_path, output_file):
    data = []
    
    # Loop through all PDF files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.pdf'):  # Process only PDF files
            pdf_path = os.path.join(folder_path, file_name)
            with pdfplumber.open(pdf_path) as pdf:
                for page in pdf.pages:
                    text = page.extract_text()
                    if text:
                        # Extract travel date
                        travel_date_match = re.search(r'\nGültigkeit: (\d{2}.\d{2}.\d{4}) 00:00 Uhr', text)
                        travel_date = travel_date_match.group(1) if travel_date_match else 'N/A'

                        # Extract train information
                        train_match = re.search(r'\nEinfache Fahrt (.*?)\nVia:', text, re.DOTALL)
                        train = train_match.group(1).strip() if train_match else 'N/A'

                        # Extract cost
                        cost_match = re.search(r'\nGesamtpreis (.*?)€', text, re.DOTALL)
                        cost = cost_match.group(1).strip() if cost_match else 'N/A'

                        # Extract Auftragsnummer
                        auftragsnummer_match = re.search(r'\nAuftragsnummer: (.*?)\nIhre', text)
                        auftragsnummer = auftragsnummer_match.group(1).strip() if auftragsnummer_match else 'N/A'

                        # Append extracted info to the data list
                        data.append({
                            'Travel Date': travel_date,
                            'Train': train,
                            'Cost (€)': cost,
                            'Auftragsnummer': auftragsnummer,
                            'File Name': file_name  # Include file name for traceability
                        })

    # Convert the data into a DataFrame
    df = pd.DataFrame(data, columns=['Travel Date', 'Train', 'Cost (€)', 'Auftragsnummer', 'File Name'])

    # Save the DataFrame to an Excel file
    df.to_excel(output_file, index=False)
    return df

In [19]:
cwd = os.getcwd()
cwd

'C:\\Users\\freed\\develop\\DB_OEBB_ticket'

In [20]:
# Example usage
folder_path = 'C:\\Users\\freed\\develop\\DB_OEBB_ticket\\new_ticket'  # Replace with the path to your folder
output_file = 'train_ticket_summary_new_ticket.xlsx'  # Replace with your desired output file name
df = extract_ticket_info_from_folder(folder_path, output_file)
